# Code for training model

In [ ]:
# Import Libraries:
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.data import transforms as T
# import some common libraries
import numpy as np
import os, json, cv2, random
from matplotlib import pyplot as plt
import yaml, copy

# import some common detectron2 utilities
from detectron2 import model_zoo, structures
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, DatasetMapper
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.utils.visualizer import ColorMode


import labelme2cocoMy

c:\Users\faraboli\AppData\Local\miniconda3\envs\bubbleid\lib\site-packages\detectron2\model_zoo\model_zoo.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
# Si on modifie les fichiers importer il faut forcer a les recharger pour que les modifs soient prises en compte
from importlib import reload
reload(labelme2cocoMy)

## Creation du fichier coco a partir des sorties de labelme

In [ ]:
# Pour le training set
labelme_folder_path_train = "dataset\\train"
coco_path_train = "train.json" # output path
labelme2cocoMy.labelme2coco(labelme_folder_path_train, coco_path_train)
# Et pour le validation set
labelme_folder_path_val = "dataset\\val"
coco_path_val = "val.json"
labelme2cocoMy.labelme2coco(labelme_folder_path_val, coco_path_val)

In [ ]:
# pour verifier le nombre de classe et leur numerotation
import json

with open("train.json") as f:
    data = json.load(f)
num_classes = len(data["categories"])
print("Nombre de classes dans JSON COCO:", num_classes)
for c in data["categories"]:
    print(c["id"], c["name"])

In [ ]:
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
val_metadata = MetadataCatalog.get("my_dataset_test")
val_dataset_dicts = DatasetCatalog.get("my_dataset_test")

Si un warning apparait c'est en general car certain label on ete fait avec circle sur labelme. Dans ce cas il faut convertir les cercles en polygone avec la fonction suivante, puis refaire le fichier coco (rerun les cellules ci dessus)

In [ ]:
# import os
# import json
# import math

# # Chemin vers le dossier contenant vos JSON
# dossier = "BubbleIDGit\\ProjetBubbleID\\training\\dataset\\val"

# # Nombre de points pour approximater le cercle en polygone
# N_POINTS = 50

# for nom_fichier in os.listdir(dossier):
#     if nom_fichier.endswith(".json"):
#         chemin_fichier = os.path.join(dossier, nom_fichier)
#         with open(chemin_fichier, "r", encoding="utf-8") as f:
#             try:
#                 data = json.load(f)
#                 shapes = data.get("shapes", [])
#                 for forme in shapes:
#                     if forme.get("shape_type") == "circle":
#                         points = forme.get("points", [])
#                         if len(points) >= 2:
#                             # On suppose points[0] = centre, points[1] = point sur le cercle
#                             cx, cy = points[0]
#                             px, py = points[1]
#                             r = math.hypot(px - cx, py - cy)

#                             # Génération des points du polygone
#                             polygon_points = [
#                                 [cx + r * math.cos(2 * math.pi * i / N_POINTS),
#                                  cy + r * math.sin(2 * math.pi * i / N_POINTS)]
#                                 for i in range(N_POINTS)
#                             ]

#                             # Remplacement des points et du type
#                             forme["points"] = polygon_points
#                             forme["shape_type"] = "polygon"

#                 # Réécriture du fichier JSON
#                 with open(chemin_fichier, "w", encoding="utf-8") as f_out:
#                     json.dump(data, f_out, indent=2)

#             except json.JSONDecodeError:
#                 print(f"Erreur lecture JSON : {nom_fichier}")

# print("Conversion des cercles en polygones terminée.")

### Visualisation des images avec annotations

In [ ]:
from matplotlib import pyplot as plt
for d in random.sample(train_dataset_dicts, 2):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.show()

## Parametres pour le training

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "./Models"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_MATLAB1.pth")  # path to the model we just trained
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # We have 1 classes.
# NOTE: this config means the number of classes, without the background. Do not use num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
#trainer.resume_or_load(resume=False) #Load a pretrained model if available (resume training) or start training from scratch if no pretrained model is available


In [ ]:
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T

def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    
    mean = 0
    std_dev = 25
    gaussian_noise = np.random.normal(mean, std_dev, image.shape).astype(np.uint8)
    #noisy_image = cv2.add(image, gaussian_noise)
    
    transform_list = [
        #T.Resize((800,600)),
        T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3),
        T.RandomSaturation(0.8, 1.4),
        #T.RandomRotation(angle=[90, 90]),
        #T.RandomNoise(mean=0.0, std=0.1),
        T.RandomLighting(0.7),
        T.RandomFlip(prob=0.4, horizontal=True, vertical=False),
    ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

trainer=CustomTrainer(cfg)
trainer.resume_or_load(resume=False)

## Start the training process

In [ ]:
trainer.train()

## Save the config to a config.yaml file

In [ ]:
config_yaml_path = "./Models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

# Test the model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

for d in random.sample(val_dataset_dicts, 1):    #select number of images for display
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=val_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image()[:,:,::-1])
    cv2.imshow(out.get_image()[:, :, ::-1])

## Average precision

In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # ou ton nombre de classes
cfg.MODEL.DEVICE = "cuda"

# Ancien modèle
cfg.MODEL.WEIGHTS = "../Customizable/model_final.pth"
predictor_old = DefaultPredictor(cfg)

# Nouveau modèle
cfg.MODEL.WEIGHTS = "Models/model_final.pth"
predictor_new = DefaultPredictor(cfg)


from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")



metrics_old = inference_on_dataset(predictor_old.model, val_loader, evaluator)
print("Ancien modèle :", metrics_old)

metrics_new = inference_on_dataset(predictor_new.model, val_loader, evaluator)
print("Nouveau modèle :", metrics_new)
